In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
{
  "background": "a lone traveler",
  "trait": "sneaky and clever",
  "location": "a dark forest",
  "goal": "find a treasure",
  "item": "a magic sword"
}

{'background': 'a lone traveler',
 'trait': 'sneaky and clever',
 'location': 'a dark forest',
 'goal': 'find a treasure',
 'item': 'a magic sword'}

In [2]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List


class GameUpdate(BaseModel):
    player_message: str = Field(description="message that the player will see")
    inventory: List[str] = Field(description="list of items in the player's inventory")
    player_stats: List[int] = Field(
        description="the player's stats (health, energy, gold)"
    )
    action_options: List[str] = Field(
        description="list of the player's action options in order"
    )
    can_rest: bool = Field(description="whether the player can rest at the moment")
    can_heal: bool = Field(description="whether the player can heal at the moment")

    generate_image: bool = Field(
        description="whether to generate an image of the game state"
    )

    image_prompt: str = Field(
        description="prompt to generate the image of the game state"
    )

    
    @validator("player_message")
    def player_message_must_be_string(cls, v):
        if not isinstance(v, str):
            raise TypeError("player_message must be a string")
        return v

    @validator("inventory")
    def inventory_must_be_list(cls, v):
        if not isinstance(v, list):
            raise TypeError("inventory must be a list")
        return v

    @validator("player_stats")
    def player_stats_must_be_list(cls, v):
        if not isinstance(v, list):
            raise TypeError("player_stats must be a list")

        # must be length 3
        if len(v) != 3:
            raise ValueError("player_stats must be length 3")
        
        # must be integers
        for stat in v:
            if not isinstance(stat, int):
                raise TypeError("player_stats must be a list of integers")
        return v

    @validator("action_options")
    def action_options_must_be_list(cls, v):
        if not isinstance(v, list):
            raise TypeError("action_options must be a list")

        # must be length 3
        if len(v) != 3:
            raise ValueError("action_options must be length 3")
        
        # must be strings
        for option in v:
            if not isinstance(option, str):
                raise TypeError("action_options must be a list of strings")
        return v
    
    @validator("can_rest")
    def can_rest_must_be_bool(cls, v):
        if not isinstance(v, bool):
            raise TypeError("can_rest must be a bool")
        return v
    
    @validator("can_heal")
    def can_heal_must_be_bool(cls, v):
        if not isinstance(v, bool):
            raise TypeError("can_heal must be a bool")
        return v
    

parser = PydanticOutputParser(pydantic_object=GameUpdate)

system_message_prompt_template = """
You are a text-based game master.
You are leading the human player through a procedurally generated text-based game.
The player has selected the following options:
    - character: {char_type}
    - location: {location}
    - items: {items}

The user is presented with very brief a description of their character and their current location.

The player is prompted with a quest to complete early on in the game.
The quest is different for each character type, location, and item selection.
The game ends when the user has completed their quest. 
As a game master, you want the player to have a fun and engaging experience.
You want the player to feel like they are in control of their character, but not so powerful that they can do anything.
You want the player to finish the quest, but you don't want it to be too easy.
The situations the player encounters should be interesting and varied, and have a fairly high chance of success.

At each point, the user is presented with a list of actions they can take.
The characters actions open-ended, but ultimitely limited, and depend on their character selection. 
For example, a mage can cast spells, but a knight cannot.
There are always 3 options. They are procedurally generated and depend on the character type, location, items, and situation.
The player can choose to take one of the actions, or to do something else.

Players can also choose to heal or rest if possible 
(e.g. the player cannot rest if they are in combat, and cannot heal if they are not injured or do not have any healing items).

The players stats are tracked throughout the game.
The stats are:
    - Health (starts at 100)
    - Energy (starts at 100)
    - Gold (starts at 0)
    
These stats are affected by the player's actions. 
Do not print these stats in the player_message, but do include them in the player_stats.
    
If the player's health reaches 0, the game ends.
If the player's energy reaches 0, the player is unable to take any actions until they rest.
If the player's gold reaches 0, the player is unable to purchase any items until they earn more gold.

The player can earn gold by completing quests, or by selling items they find.
The player can spend gold on items that will help them complete their quest.

In certain exciting situations, the you should recomend that the game generate an image of the game state.
For example, if the player comes upon a new enemy, or a breathtaking new location.
The image_prompt is a string that will be passed to the image generator.
It should be a short description of the visual elements of the game state. It should not contain any information that is not already in the player_message.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(
    system_message_prompt_template
)

# #####################


human_template = "Respond to the players input.\n{format_instructions}\n{input}"
human_message_prompt_template = HumanMessagePromptTemplate.from_template(
    human_template,
)

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from IPython.display import Image

items = ["laser baton", "magnetic shoes", "static grenade"]
location = "abandoned building"
char_type = "cyborg"

prompt = ChatPromptTemplate(
    messages=[
        system_message_prompt.format(
            char_type=char_type, location=location, items=items
        ),
        MessagesPlaceholder(variable_name="history"),
        human_message_prompt_template,
    ],
    
    input_variables=["history", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chat_llm = ChatOpenAI(temperature=1)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=chat_llm)

response = conversation.predict(input="Let's Start")
game_update = parser.parse(response)


In [4]:
response = conversation.predict(input=game_update.action_options[0])
game_update = parser.parse(response)


In [4]:
print(game_update.player_message)

Welcome to the abandoned building. Your mission is to retrieve a stolen prototype from the top floor. There are security drones patrolling the area. Be careful.


In [6]:
game_update.action_options

['Search the area', 'Hack into the security system', 'Proceed with caution']

In [7]:
print(response)

{
  "player_message": "Welcome to the abandoned building! As a cyborg, you are equipped with a laser baton, magnetic shoes, and a static grenade. Your goal is to locate and extract a valuable piece of technology hidden deep within the building.",
  "inventory": [
    "laser baton",
    "magnetic shoes",
    "static grenade"
  ],
  "player_stats": [100, 100, 0],
  "action_options": [
    "Search the area",
    "Hack into the security system",
    "Proceed with caution"
  ],
  "can_rest": true,
  "can_heal": false,
  "generate_image": false,
  "image_prompt": ""
}


In [8]:
import openai
response = openai.Image.create(
  prompt=game_update.image_prompt,
  n=1,
  size="1024x1024",
)
image_url = response['data'][0]['url']
image_url


'https://oaidalleapiprodscus.blob.core.windows.net/private/org-3MkXL2eodP9fG13iG794QjSp/user-8DyY6pOUFDlAHg0begb8F7k7/img-TTVNM9cW1c96240SlIBjb4aP.png?st=2023-06-18T03%3A22%3A17Z&se=2023-06-18T05%3A22%3A17Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-06-17T20%3A53%3A54Z&ske=2023-06-18T20%3A53%3A54Z&sks=b&skv=2021-08-06&sig=NWcN992rm/c8H9S7s3%2BraXn/dA2JH3FUd8xHaWKOtzM%3D'